In [25]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [8]:
def fetch_data_from_api(url, params=None):
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.text}")
        return {}

def get_quotes(symbol, api_key):
    quote_url = f"https://finnhub.io/api/v1/quote?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(quote_url)

def get_financials_reported(symbol, api_key):
    financial_url = f"https://finnhub.io/api/v1/stock/financials-reported?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(financial_url)

def get_usa_spending(symbol, api_key):
    spending_url = f"https://finnhub.io/api/v1/stock/usa-spending?symbol={symbol}&token={api_key}"
    return fetch_data_from_api(spending_url)

In [9]:
def extract_financial_data(data, section, concepts=None):
    financial_data = []
    if 'data' in data:
        for report in data['data']:
            if 'report' in report and section in report['report']:
                section_data = report['report'][section]
                if concepts:  # Filter data based on the list of important concepts
                    section_data = [item for item in section_data if item['concept'] in concepts]
                financial_data.extend(section_data)
    return financial_data


In [26]:
# Load environment variables from .env file
load_dotenv()

# Define your connection parameters
username = os.getenv('username')  # Your RDS username
password = os.getenv('password')  # Your RDS password
host = os.getenv('host')  # Your RDS endpoint
port = os.getenv('port')  # Default MySQL port
database = os.getenv('database')  # The name of the database you want to create

api_key = os.getenv('FINNHUB_API_KEY')  # Replace with your actual API key

# Create an engine to connect to MySQL Database using a hardcoded URL for testing
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=364)
formatted_start_date = datetime.strptime(start_date.strftime('%Y-%m-%d'), '%Y-%m-%d')
formatted_end_date = datetime.strptime(end_date.strftime('%Y-%m-%d'), '%Y-%m-%d')

company_symbols = ["JPM", "GS", "C", "JLL", "DIS", "TPR", "F", "XOM", "AAPL", "AMZN", "PFE", "MRK"]
concepts = ['us-gaap_AssetsCurrent', 'us-gaap_AssetsNoncurrent', 'us-gaap_Assets',
            'us-gaap_LiabilitiesCurrent', 'us-gaap_LiabilitiesNoncurrent', 'us-gaap_Liabilities',
            'us-gaap_StockholdersEquity', 'us-gaap_CommonStocksIncludingAdditionalPaidInCapital',
            'us-gaap_RetainedEarningsAccumulatedDeficit']

quotes_data = []
financials_data = []
spending_data = []

for symbol in company_symbols:
    # Fetch and process quotes
    quote = get_quotes(symbol, api_key)
    if quote:
        quote['symbol'] = symbol
        quotes_data.append(quote)

    # Fetch and process financials
    financial_data = get_financials_reported(symbol, api_key)
    bs_data = extract_financial_data(financial_data, 'bs', concepts)
    for item in bs_data:
        item['symbol'] = symbol
    financials_data.extend(bs_data)

    # Fetch and process spending data
    spend_data = get_usa_spending(symbol, api_key)
    if 'data' in spend_data:
        for spend in spend_data['data']:
            spend_date = pd.to_datetime(spend['actionDate'])
            if formatted_start_date <= spend_date <= formatted_end_date:
                spend['symbol'] = symbol
                spending_data.append(spend)

    time.sleep(1)  # Respect API rate limits

# Convert lists to DataFrames
quotes_df = pd.DataFrame(quotes_data)
financials_df = pd.DataFrame(financials_data)
spending_df = pd.DataFrame(spending_data)

# Upload each DataFrame to the MySQL database
quotes_df.to_sql(name='quotes', con=engine, if_exists='replace', index=False)
financials_df.to_sql(name='financials_reported', con=engine, if_exists='replace', index=False)
spending_df.to_sql(name='usa_spending', con=engine, if_exists='replace', index=False)

print("Data uploaded successfully.")

Data uploaded successfully.


In [24]:
# Load environment variables from .env file
load_dotenv()

# Define your connection parameters
username = os.getenv('username')  # Your RDS username
password = os.getenv('password')  # Your RDS password
host = os.getenv('host')  # Your RDS endpoint
port = os.getenv('port')  # Default MySQL port
database = os.getenv('database')  # The name of the database you want to create

# Connect to the database without specifying the database name first
# This checks if your instance connection is fine
test_engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}')

try:
    with test_engine.connect() as test_connection:
        print("RDS instance connection successful!")
except Exception as e:
    print(f"Failed to connect to RDS instance: {e}")

# If the instance connection is successful, connect to your specific database
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

try:
    with engine.connect() as connection:
        print(f"Connected successfully to the database '{database}'!")
except Exception as e:
    print(f"Failed to connect to the database '{database}': {e}")



RDS instance connection successful!
Connected successfully to the database 'mayah_bosworth_sql_project'!
